In [12]:
import utils

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, \
    TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score

import keras
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from keras.optimizers import RMSprop, Adam, SGD

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy

from os.path import join, basename, exists
from os import makedirs, listdir

In [3]:
def preproc_target_train(data, reduce_classes=False):
    diagnoses = data['Код_диагноза'].copy()
    
    if reduce_classes:
        pop_diagnoses = set(utils.get_most_popular_diagnoses(diagnoses, percent=.80))
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    else:
        pop_diagnoses = set(diagnoses)
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def preproc_target_test(data, pop_diagnoses, most_pop_diagnose):
    diagnoses = data['Код_диагноза'].copy()
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def join_topics(data, topics):
    data = data.copy()
    
    topics_df = pd.DataFrame(dict(zip(
        ['topic' + str(i) for i in range(topics.shape[1])],
        [topics[:, i] for i in range(topics.shape[1])])))
    topics_df['Id_Записи'] = data['Id_Записи']

    data = data.join(topics_df, on='Id_Записи', rsuffix='_topics', how='outer')
    data = data.drop(columns=['Id_Записи_topics'])
    
    return data

In [21]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class DoctorsPopularityTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        doctors = x.fillna('sss')
        doctors_voc, counts = np.unique(doctors, return_counts=True)
        self.pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
        
        return self

    def transform(self, x):
        x = x.fillna('sss')
        x[x == 'sss'] = self.pop_doctor
        
        return x
    
class GenderTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.copy()
        x[x == 1] = 0
        x[x == 2] = 1
        
        return np.expand_dims(x, axis=1)
    
class AgeTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)
    
class TopicsTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return x

class ClinicTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        clinics, counts = np.unique(x, return_counts=True)
        self.clinics_to_idx = dict(zip(clinics, range(len(clinics))))
        
        self.most_pop_clinic_idx = np.argmax(counts)
        
        return self
    
    def transform(self, x):
        x_vec = np.zeros((x.shape[0], len(self.clinics_to_idx)), dtype=np.float32)
        for i, clinic in enumerate(x):
            if clinic in self.clinics_to_idx:
                x_vec[i, self.clinics_to_idx[clinic]] = 1.
            else:
                x_vec[i, self.most_pop_clinic_idx] = 1.
        
        return x_vec
    
class RepeatsTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)
    
class PatientHistoryTransformator(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=30):
        self.n_features = n_features
    
    def fit(self, x, y=None):
        diagnoses_concat = x.groupby('Id_Пациента').apply(
            lambda group: ' '.join(group['Код_диагноза'].values)
        )

        vectorizer = HashingVectorizer(n_features=self.n_features)
        diagnoses_hashed = vectorizer.fit_transform(diagnoses_concat)
        
        self.history = pd.DataFrame(data={
            'Id_Пациента': diagnoses_concat.index
        })

        self.features = ['history_{}'.format(i) for i in range(self.n_features)]
        self.history[self.features] = pd.DataFrame(diagnoses_hashed.toarray())
        
        return self
    
    def transform(self, x):
        merged = pd.merge(x, self.history, how='left')
        x_history = merged[self.features].fillna(0.0)
        
        return x_history

In [22]:
class NNModel(BaseEstimator, TransformerMixin):
    def __init__(self, batch_size, epochs, tb_log_dir, model_path, validation_split=0.3):
        self.batch_size = batch_size
        self.epochs = epochs
        self.tb_log_dir = tb_log_dir
        self.model_path = model_path
        self.validation_split = validation_split
    
    def fit(self, x, y=None):
        self.classes = np.unique(y)
        self.classes_voc = dict(zip(self.classes, range(self.classes.shape[0])))
        self.voc_classes = dict(zip(range(self.classes.shape[0]), self.classes))
        y_proc = np.zeros((y.shape[0], self.classes.shape[0]), dtype=np.float32)
        for i, yc in enumerate(y):
            y_proc[i, self.classes_voc[yc]] = 1.
        
        self.model = Sequential([
            InputLayer(input_shape=(x.shape[1],)),
            Dense(2048, activation='sigmoid'),
            Dense(2048, activation='sigmoid'),
#             Dense(4096, activation='sigmoid'),
#             Dense(4096, activation='sigmoid'),
            Dense(self.classes.shape[0], activation='softmax')
        ])
        
        optim = RMSprop(lr=1e-4, decay=1e-6)
        self.model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model.fit(x, y_proc,
                  batch_size=self.batch_size, epochs=self.epochs,
                  callbacks=[
                      TensorBoard(log_dir=self.tb_log_dir, batch_size=self.batch_size),
                      ModelCheckpoint(filepath=self.model_path, monitor='acc', period=5)
                  ],
                  validation_split=self.validation_split)
        
        return self
        
    def transform(self, x):
        pred = self.model.predict(x)
        max_idxs = np.argmax(pred, axis=1)
        
        return np.array(list(map(lambda max_idx: self.voc_classes[max_idx], max_idxs)))

In [23]:
def create_pipeline(experiment_dir, model_fname_prefix, no_validation=False, validation_split=0.3):
    model_name = model_fname_prefix + str(utils.get_next_model_id(experiment_dir))
    tb_log_dir = join(experiment_dir, 'log', model_name)
    models_dir = join(experiment_dir, 'models')

    if no_validation:
        model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=True)
        validation_split = 0.0
    else:
        model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=False)

    pipe = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('complaints_pipe', Pipeline([
                    ('complaint_selector', ItemSelector(key='Жалобы (unigramm)')),
                    ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=3, stop_words=stopwords.words('russian')))
                ])),
                ('complaints_n_pipe', Pipeline([
                    ('complaint_n_selector', ItemSelector(key='Жалобы (ngramm)')),
                    ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=1, stop_words=stopwords.words('russian')))
                ])),
                ('doctor_pipe', Pipeline([
                    ('doctor_selector', ItemSelector(key='Врач')),
                    ('doc_pop', DoctorsPopularityTransformator()),
                    ('count_vect', CountVectorizer())
                ])),
                ('gender_pipe', Pipeline([
                    ('gender_selector', ItemSelector(key='Пол')),
                    ('gender_transform', GenderTransformator())
                ])),
                ('age_pipe', Pipeline([
                    ('age_selector', ItemSelector(key='Возраст')),
                    ('age_transformator', AgeTransformator())
                ])),
                ('topics_pipe', Pipeline([
                    ('topic_selector', ItemSelector(key=['topic' + str(i) for i in range(355)])),
                    ('topics_transform', TopicsTransformator())
                ])),
                ('clinic_pipe', Pipeline([
                    ('clinic_selector', ItemSelector(key='Клиника')),
                    ('clinic_transform', ClinicTransformator())
                ])),
                ('repeats_pipe', Pipeline([
                    ('repeats_selector', ItemSelector(key='Повторный приём')),
                    ('repeats_transform', RepeatsTransformator())
                ])),
                ('history_pipe', PatientHistoryTransformator(n_features=50))
            ]
        )),
        ('clf', NNModel(batch_size=128, epochs=200, tb_log_dir=tb_log_dir,
                        model_path=model_path, validation_split=validation_split))
    ])
    
    return pipe, model_name

In [24]:
pipe, model_name = create_pipeline('simple_models2', 'nn_3dense', no_validation=False, validation_split=0.3)

In [25]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = utils.join_topics(train, train_topics)
# train, valid = train_test_split(train, test_size=0.3)

In [26]:
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)
# valid_y, _, _ = preproc_target_test(valid, pop_diagnoses, most_pop_diagnose)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [27]:
np.unique(train_y).shape

(2302,)

In [ ]:
pipe.fit(train, train_y)

Train on 43383 samples, validate on 18593 samples
Epoch 1/200
43383/43383 [==============================] - 9s 204us/step - loss: 5.6789 - acc: 0.0522 - val_loss: 6.1825 - val_acc: 0.0525
Epoch 2/200
15104/43383 [=========>....................] - ETA: 4s - loss: 5.4564 - acc: 0.0757

In [18]:
pred = pipe.transform(train)

In [19]:
np.unique(pred, return_counts=True)

(array(['A46', 'A63.0', 'B00', 'B00.1', 'B00.8', 'B02', 'B07', 'B08.1',
        'B35.1', 'B35.3', 'B35.6', 'B36.0', 'B37', 'B37.3+', 'B97.1',
        'B97.7', 'D17.0', 'D17.1', 'D17.2', 'D18.0', 'D23', 'D23.3',
        'D23.5', 'D23.9', 'D24', 'D25', 'D50', 'D50.9', 'D89.8', 'E01.1',
        'E01.8', 'E03.8', 'E03.9', 'E04.1', 'E04.2', 'E05.0', 'E06.3',
        'E11', 'E22.1', 'E28', 'E28.1', 'E28.8', 'E28.9', 'E66.0', 'E89.0',
        'F41.0', 'F41.2', 'F45.3', 'F95.0', 'G24', 'G24.9', 'G43.0',
        'G44.2', 'G50.0', 'G51.0', 'G55*', 'G55.1*', 'G55.3*', 'G56.0',
        'G56.2', 'G90', 'G90.8', 'G90.9', 'G93.4', 'H00', 'H00.0', 'H00.1',
        'H01.0', 'H04.1', 'H10.0', 'H10.1', 'H10.2', 'H10.3', 'H10.5',
        'H10.8', 'H11.3', 'H15.1', 'H16.1', 'H16.2', 'H20.0', 'H35.0',
        'H35.3', 'H35.4', 'H40.1', 'H52.0', 'H52.1', 'H52.2', 'H52.4',
        'H52.5', 'H60', 'H61.2', 'H65', 'H65.0', 'H65.1', 'H66.0', 'H66.1',
        'H68.0', 'H68.1', 'H81.1', 'H81.2', 'H90.3', 'H93.1', 

## Submit

In [9]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = join_topics(train, train_topics)
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [10]:
pipe.fit(train, train_y)

Epoch 1/100
61976/61976 [==============================] - 13s 215us/step - loss: 5.8733 - acc: 0.0998
Epoch 2/100
61976/61976 [==============================] - 13s 212us/step - loss: 5.5176 - acc: 0.1808
Epoch 3/100
61976/61976 [==============================] - 13s 211us/step - loss: 5.1080 - acc: 0.2237
Epoch 4/100
61976/61976 [==============================] - 13s 212us/step - loss: 4.7338 - acc: 0.2444
Epoch 5/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.4498 - acc: 0.2577
Epoch 6/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.2481 - acc: 0.2679
Epoch 7/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.1090 - acc: 0.2773
Epoch 8/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.0043 - acc: 0.2860
Epoch 9/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.9136 - acc: 0.2952
Epoch 10/100
61976/61976 [==============================] - 13s 211us/ste

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('complaints_pipe', Pipeline(memory=None,
     steps=[('complaint_selector', ItemSelector(key='Жалобы (unigramm)')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, e...:48_2018/{epoch:02d}_{acc:.2f}.h5',
    tb_log_dir='simple_models/log/nn14', validation_split=0.0))])

In [14]:
test = utils.load_data('data/test_data_complaints_repeats_doctors.csv')
test_topics = np.load('data/topics_test_ngramm.npy')
test = utils.join_topics(test, test_topics)

In [15]:
test_pred = pipe.transform(test)

In [16]:
submit = pd.DataFrame({'Id_Записи': test['Id_Записи'], 'Код_диагноза': test_pred})

In [17]:
submit['Код_диагноза'].value_counts()

M42.1     1891
N76.0     1689
J06.9     1413
Z00.0      970
N41.1      929
Z32.1      903
J00        892
M54.2      839
I11        605
K30        605
N77.1*     584
J35.0      532
M53.8      523
H52.1      504
Z34.0      471
K29.9      449
M54.8      431
N60.1      416
Z01.8      369
D23.9      359
J01.0      319
N40        318
G90        273
M65        271
M79.1      269
D25        234
H35.0      232
N72        229
I83.9      212
M51.3      209
          ... 
H90.3        2
N70.1        2
M75.3        2
H20.0        2
I15          2
M47.8        2
Z00.1        1
E28.9        1
N11.0        1
N43.4        1
D22.5        1
K04.0        1
L23.9        1
E01.8        1
H11.3        1
K29.6        1
H65.1        1
H01.0        1
N34          1
L08.8        1
H15.1        1
M16          1
E14          1
D18.0        1
D17.1        1
T15.1        1
S20.2        1
M71.3        1
M42.0        1
K26          1
Name: Код_диагноза, Length: 321, dtype: int64

In [18]:
submit.to_csv('submit/bow_simple2_nn2_3dense_diag_all_compl_uni-n_gram_doctor_topics_clinic_repeats_history.csv', header=True, index=False)